In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('6Exercises').master('local[*]').getOrCreate()
spark.conf.set('spark.sql.shuffle.partitions', 8)

In [3]:
import os
os.listdir(os.getcwd())

['sellers_parquet',
 '.DS_Store',
 'Untitled.ipynb',
 'sales_parquet',
 '.ipynb_checkpoints',
 'products_parquet']

In [4]:
df_products = spark.read.parquet('products_parquet')

In [5]:
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)



In [6]:
df_sales = spark.read.parquet('sales_parquet')

In [7]:
df_sales.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)



In [8]:
df_sellers = spark.read.parquet('sellers_parquet')

In [9]:
df_sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: string (nullable = true)



# Find out how many orders, how many products and how many sellers are in the data.

In [10]:
order_cnt = df_sales.select('order_id').count()
product_cnt = df_products.select('product_id').count()
seller_cnt = df_sellers.select('seller_id').count()
print(f'Order count: {order_cnt}, Product count: {product_cnt}, Seller count: {seller_cnt}')

Order count: 20000040, Product count: 75000000, Seller count: 10


# How many products have been sold at least once? Which is the product contained in more orders?

In [11]:
df_sales.select('product_id').distinct().count()

993429

In [12]:
df_sales.groupBy('product_id').count().orderBy('count', ascending = False).show(1)

+----------+--------+
|product_id|   count|
+----------+--------+
|         0|19000000|
+----------+--------+
only showing top 1 row



# How many distinct products have been sold in each day?

In [13]:
df_sales.select('date', 'product_id').distinct().groupby('date').count().show()

+----------+------+
|      date| count|
+----------+------+
|2020-07-06|100765|
|2020-07-01|100337|
|2020-07-02| 99807|
|2020-07-07| 99756|
|2020-07-10| 98973|
|2020-07-04| 99791|
|2020-07-09|100501|
|2020-07-08| 99662|
|2020-07-03|100017|
|2020-07-05| 99796|
+----------+------+



# What is the average revenue of the orders?

In [22]:
from pyspark.sql.functions import *
df_sales.select('order_id', 'product_id', 'num_pieces_sold').join(df_products.select('product_id', 'price'), on= 'product_id', how='inner').withColumn('total_price', col('num_pieces_sold')*col('price')).select(mean('total_price')).show()

+------------------+
|  avg(total_price)|
+------------------+
|1246.1338560822878|
+------------------+



23/03/15 18:52:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 286093 ms exceeds timeout 120000 ms
23/03/15 18:52:32 WARN SparkContext: Killing executors is not supported by current scheduler.
